## IRT で計算される数値を取り込む方法

 IRT では、項目ごとの識別パラメータと困難度パラメータが得られるとともに、各受験者の特性を示す能力パラメータが得られる。ここでは、2PLモデルに基づく各種パラメータが、R の ltm パッケージにより計算された後に、それらのパラメータをプログラムで利用な形で得る方法についてまとめる。

### IRT 分析のためのライブラリ (ltm) の読み込み

In [1]:
require(ltm)

Loading required package: ltm
Loading required package: MASS
Loading required package: msm
Loading required package: polycor
Loading required package: mvtnorm
Loading required package: sfsmisc


### データファイルの読み込みと分析

CSV形式のデータファイルを読み込み、データを格納するとともに、IRTの分析を行う。

* data : 読み込んだデータ  
* result : ltm による分析結果  
* result_summary : ltm による分析結果のサマリー形式  
* result_score : factor.scores() による各受験者の能力値の結果  

In [2]:
data <- read.csv('~/IrtWeb/irt_sample.csv', header=TRUE)

result <- ltm(data~z1, IRT.param=TRUE)
result_summary <- summary(result)
result_score <- factor.scores(result, resp.pattern=data)

### ltm の結果から識別度を得る

result の中に、困難度、識別度が入っていることがわかる。

In [3]:
result


Call:
ltm(formula = data ~ z1, IRT.param = TRUE)

Coefficients:
   Dffclt  Dscrmn
A  -1.176   1.216
B  -0.392   1.574
C  -0.053  16.899
D   0.201   1.170
E   1.159   1.181

Log.Lik: -56.733


上の表 および str(result) の結果から、困難度および識別度は、result の中の coefficients に入っていそうである。

In [4]:
str(result)

List of 14
 $ coefficients: num [1:5, 1:2] 1.43 0.616 0.894 -0.235 -1.369 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:5] "A" "B" "C" "D" ...
  .. ..$ : chr [1:2] "(Intercept)" "z1"
 $ log.Lik     : num -56.7
 $ convergence : int 0
 $ hessian     : num [1:10, 1:10] 2.786 -0.247 -0.232 -0.193 -0.134 ...
 $ counts      : Named int [1:2] 2 1
  ..- attr(*, "names")= chr [1:2] "function" "gradient"
 $ patterns    :List of 2
  ..$ X  : num [1:12, 1:5] 0 0 0 0 1 1 1 1 1 1 ...
  ..$ obs: int [1:12] 2 1 1 1 3 1 1 2 2 1 ...
 $ GH          :List of 2
  ..$ Z  : num [1:21, 1:2] 1 1 1 1 1 1 1 1 1 1 ...
  ..$ GHw: num [1:21] 2.10e-14 4.98e-11 1.45e-08 1.23e-06 4.22e-05 ...
 $ max.sc      : num 0.000346
 $ ltst        :List of 5
  ..$ factors: int 1
  ..$ inter  : logi FALSE
  ..$ quad.z1: logi FALSE
  ..$ quad.z2: logi FALSE
  ..$ nams   : chr [1:2] "(Intercept)" "z1"
 $ X           :'data.frame':	20 obs. of  5 variables:
  ..$ A: int [1:20] 0 0 0 1 1 1 1 1 1 0 ...
  ..$ B: int [1:20] 0

result$coefficients でアクセスしたところ、識別度 (z1) は入手できるが、困難度とセットになっていない（表示されている Intercept の値は困難度ではない）。

In [5]:
result$coefficients

,(Intercept),z1
A,1.430019,1.215814
B,0.6164858,1.5742688
C,0.8940602,16.8993942
D,-0.2354762,1.1696718
E,-1.368762,1.181162


In [6]:
result$coefficient[,2]

A         B         C         D         E 
 1.215814  1.574269 16.899394  1.169672  1.181162

識別度だけは入手できるので、変数 a に入れておく。

In [7]:
a <- result$coefficient[,2]

また、項目名、そして項目数を入手しておく。

In [31]:
item_names <- rownames(result$coefficient)
nitem <- length(item_names)

In [32]:
print(nitem)
print(item_names)
print(a)

[1] 5
[1] "A" "B" "C" "D" "E"
        A         B         C         D         E 
 1.215814  1.574269 16.899394  1.169672  1.181162 


### ltm の結果から困難度を得る

summary 情報から困難度を得ることを考える。

In [21]:
result_summary


Call:
ltm(formula = data ~ z1, IRT.param = TRUE)

Model Summary:
   log.Lik      AIC      BIC
 -56.73256 133.4651 143.4224

Coefficients:
           value  std.err  z.vals
Dffclt.A -1.1762   0.7697 -1.5281
Dffclt.B -0.3916   0.4449 -0.8801
Dffclt.C -0.0529   0.8234 -0.0642
Dffclt.D  0.2013   0.5044  0.3991
Dffclt.E  1.1588   0.8258  1.4034
Dscrmn.A  1.2158   0.9136  1.3308
Dscrmn.B  1.5743   1.0456  1.5056
Dscrmn.C 16.8994 260.5850  0.0649
Dscrmn.D  1.1697   0.8446  1.3849
Dscrmn.E  1.1812   0.9753  1.2111

Integration:
method: Gauss-Hermite
quadrature points: 21 

Optimization:
Convergence: 0 
max(|grad|): 0.00035 
quasi-Newton: BFGS 


識別度、困難度が表になっており、 $coefficients でアクセスしてみる。

In [22]:
result_summary$coefficients

,value,std.err,z.vals
Dffclt.A,-1.1761828,0.7697059,-1.5280937
Dffclt.B,-0.3916013,0.4449440,-0.8801138
Dffclt.C,-0.05290487,0.82343727,-0.06424881
Dffclt.D,0.2013182,0.5043902,0.3991319
Dffclt.E,1.1588266,0.8257548,1.4033543
Dscrmn.A,1.2158138,0.9135998,1.3307947
Dscrmn.B,1.574269,1.045622,1.505581
Dscrmn.C,16.89939422,260.58495015,0.06485177
Dscrmn.D,1.1696718,0.8445942,1.3848921
Dscrmn.E,1.1811624,0.9752508,1.2111371


ここから、識別度、困難度を入手できそうである。ひとまず、困難度を変数 b に入れる。

表は１列目（value のところ）に、困難度と識別度が連続しているので、１行目から項目数だけ取り出すと困難度だけが入手できるはずである。

In [33]:
result_summary$coefficients[,1][1:nitem]

Dffclt.A    Dffclt.B    Dffclt.C    Dffclt.D    Dffclt.E 
-1.17618278 -0.39160134 -0.05290487  0.20131818  1.15882655

In [34]:
b <- result_summary$coefficients[,1][1:5]

識別度、困難度が得られたので、表を作成しておく。

In [51]:
params <- data.frame(a, b)
params

,a,b
A,1.215814,-1.176183
B,1.574269,-0.3916013
C,16.89939,-0.05290487
D,1.169672,0.2013182
E,1.181162,1.158827


### 能力値の産出結果から、各受験者の能力値を得る

In [36]:
result_score


Call:
ltm(formula = data ~ z1, IRT.param = TRUE)

Scoring Method: Empirical Bayes

Factor-Scores for specified response patterns:
   A B C D E Obs   Exp     z1 se.z1
1  0 0 0 0 0   2 2.019 -1.201 0.698
2  0 0 0 0 0   2 2.019 -1.201 0.698
3  0 0 0 1 0   1 0.422 -0.670 0.655
4  1 0 0 0 0   3 2.136 -0.650 0.653
5  1 0 1 1 0   1 0.676  0.134 0.272
6  1 0 0 0 0   3 2.136 -0.650 0.653
7  1 1 0 0 0   2 1.235 -0.247 0.283
8  1 1 1 1 0   3 2.893  0.529 0.678
9  1 0 0 0 0   3 2.136 -0.650 0.653
10 0 1 1 0 0   1 0.303  0.090 0.204
11 1 1 1 1 0   3 2.893  0.529 0.678
12 1 1 0 0 0   2 1.235 -0.247 0.283
13 1 1 1 0 0   2 1.854  0.172 0.344
14 1 1 1 1 0   3 2.893  0.529 0.678
15 1 1 1 0 0   2 1.854  0.172 0.344
16 1 1 1 1 1   2 2.354  1.112 0.728
17 1 1 1 0 1   1 0.940  0.535 0.678
18 1 0 1 1 1   1 0.303  0.358 0.631
19 0 1 0 1 1   1 0.035 -0.186 0.193
20 1 1 1 1 1   2 2.354  1.112 0.728


アクセス方法を探るために str() 関数を用いると、$score.dat で値が取れそうである。

In [37]:
str(result_score)

List of 6
 $ score.dat:'data.frame':	20 obs. of  9 variables:
  ..$ A    : num [1:20] 0 0 0 1 1 1 1 1 1 0 ...
  ..$ B    : num [1:20] 0 0 0 0 0 0 1 1 0 1 ...
  ..$ C    : num [1:20] 0 0 0 0 1 0 0 1 0 1 ...
  ..$ D    : num [1:20] 0 0 1 0 1 0 0 1 0 0 ...
  ..$ E    : num [1:20] 0 0 0 0 0 0 0 0 0 0 ...
  ..$ Obs  : num [1:20] 2 2 1 3 1 3 2 3 3 1 ...
  ..$ Exp  : num [1:20] 2.019 2.019 0.422 2.136 0.676 ...
  ..$ z1   : num [1:20] -1.201 -1.201 -0.67 -0.65 0.134 ...
  ..$ se.z1: num [1:20] 0.698 0.698 0.655 0.653 0.272 ...
 $ method   : chr "EB"
 $ B        : num 5
 $ call     : language ltm(formula = data ~ z1, IRT.param = TRUE)
 $ resp.pats: logi TRUE
 $ coef     : num [1:5, 1:2] -1.1762 -0.3916 -0.0529 0.2013 1.1588 ...
  ..- attr(*, "dimnames")=List of 2
  .. ..$ : chr [1:5] "A" "B" "C" "D" ...
  .. ..$ : chr [1:2] "Dffclt" "Dscrmn"
 - attr(*, "class")= chr "fscores"


In [39]:
head(result_score$score.dat)

,A,B,C,D,E,Obs,Exp,z1,se.z1
1,0,0,0,0,0,2,2.019466,-1.201139,0.6975804
2,0,0,0,0,0,2,2.019466,-1.201139,0.6975804
3,0,0,0,1,0,1,0.4217768,-0.6698989,0.654864
4,1,0,0,0,0,3,2.13579,-0.6501529,0.6534604
5,1,0,1,1,0,1,0.6763016,0.1341029,0.2724876
6,1,0,0,0,0,3,2.13579,-0.6501529,0.6534604


上の表で、8列目（可変である項目数＋3列目）のところに 'z1' があって、これが能力値なので、この値を取り出して変数 z1 に入れる。

In [47]:
z1 <- result_score$score.dat[,nitem+3]

In [48]:
z1

[1] -1.20113889 -1.20113889 -0.66989886 -0.65015292  0.13410295 -0.65015292
 [7] -0.24654444  0.52929460 -0.65015292  0.08959069  0.52929460 -0.24654444
[13]  0.17186145  0.52929460  0.17186145  1.11229049  0.53457565  0.35768848
[19] -0.18618437  1.11229049

### 各受験者が各項目で正解する確率

各項目の識別度 a、困難度 b が出たので、それと各受験者の能力値を用いると、項目ごとに各受験者が正解する確率を計算できる。

まず、2PL モデルに基づく確率計算のための関数を設定する。

$$p = \frac{1}{1 + e^{-1.701 a (x - b)}}$$

In [52]:
model2pl <- function(x, a, b){
  return (1.0 / (1.0 + exp(-1.701 * a * (x - b))))
}

確率の計算は、受験者数×項目数 の行列として算出するものとする。これは、もともとの CSV ファイルの内容と同じ大きさ dim(data) である。

受験者数を nperson, 項目数を nitem に入れておく。

In [57]:
print(dim(data))

nperson <- dim(data)[1]
nitem <- dim(data)[2]

[1] 20  5


受験者数×項目数 の行列を作成し、計算した確率を入れていく。

In [58]:
mat <- matrix(nrow=nperson, ncol=nitem)

for (i in 1:nperson) {
    for (j in 1:nitem) {
        mat[i, j] <- model2pl(z1[i], a[j], b[j])
    }
}

In [66]:
head(mat)

4.870999e-01,1.026791e-01,4.626455e-15,5.784814e-02,8.649714e-03
4.870999e-01,1.026791e-01,4.626455e-15,5.784814e-02,8.649714e-03
7.402071e-01,3.218607e-01,1.983081e-08,1.501553e-01,2.474207e-02
7.479827e-01,3.335088e-01,3.498261e-08,1.552378e-01,2.571763e-02
0.9376025,0.8034104,0.9953936,0.4666167,0.1131631
7.479827e-01,3.335088e-01,3.498261e-08,1.552378e-01,2.571763e-02


data.frame 型に変換して、項目名を設定する。項目名は、もともとの項目名に加えて推測値であることがわかるように名前を拡張した。

In [70]:
expmat <- data.frame(mat)
names(expmat) <- paste(item_names, rep(".exp", 5))

In [71]:
head(expmat)

,A .exp,B .exp,C .exp,D .exp,E .exp
1,0.4870999,0.1026791,4.626455e-15,0.05784814,0.008649714
2,0.4870999,0.1026791,4.626455e-15,0.05784814,0.008649714
3,0.7402071,0.3218607,1.983081e-08,0.1501553,0.02474207
4,0.7479827,0.3335088,3.498261e-08,0.1552378,0.02571763
5,0.9376025,0.8034104,0.9953936,0.4666167,0.1131631
6,0.7479827,0.3335088,3.498261e-08,0.1552378,0.02571763


各被験者の各問題に対する答えが正解かどうか（答えが正解であれば1、不正解であれば0）と、
正解となる確率の差を data.frame 型の変数 diffans に残す。（被験者数×項目数）

正解する確率が 0.7 なのに、不正解 0 であれば、差は -0.7 となる。

* 符号が正（0.5以上は特に）：間違ってしまう確率が高いのに正解だった。  
* 符号が負（-0.5以下は特に）：正解する確率が高い（正解していい）はずなのに間違った。  
* 差が -0.5 〜 0.5 の範囲 ： （被験者の能力値からみて）正解を出す確率が示すとおりの正解／不正解の結果となった。

In [77]:
diffans <- data.frame(data - expmat)
names(diffans) <- paste(item_names, rep(".diff", 5))

In [79]:
head(diffans)

,A .diff,B .diff,C .diff,D .diff,E .diff
1,-0.4870999,-0.1026791,-4.626455e-15,-0.05784814,-0.008649714
2,-0.4870999,-0.1026791,-4.626455e-15,-0.05784814,-0.008649714
3,-0.7402071,-0.3218607,-1.983081e-08,0.8498447,-0.02474207
4,0.2520173,-0.3335088,-3.498261e-08,-0.1552378,-0.02571763
5,0.0623975,-0.8034104,0.004606351,0.5333833,-0.1131631
6,0.2520173,-0.3335088,-3.498261e-08,-0.1552378,-0.02571763


もともとの解答（正解、不正解データ）と上で計算した各項目ごとの正解確率、（正解、不正解）の０／１データと正解確率との差の表、そして各受験者の能力値を結合させた表を作成する。

In [82]:
dataall <- cbind(data, expmat, diffans, z1)

In [83]:
head(dataall)

,A,B,C,D,E,A .exp,B .exp,C .exp,D .exp,E .exp,A .diff,B .diff,C .diff,D .diff,E .diff,z1
1,0,0,0,0,0,0.4870999,0.1026791,4.626455e-15,0.05784814,0.008649714,-0.4870999,-0.1026791,-4.626455e-15,-0.05784814,-0.008649714,-1.201139
2,0,0,0,0,0,0.4870999,0.1026791,4.626455e-15,0.05784814,0.008649714,-0.4870999,-0.1026791,-4.626455e-15,-0.05784814,-0.008649714,-1.201139
3,0,0,0,1,0,0.7402071,0.3218607,1.983081e-08,0.1501553,0.02474207,-0.7402071,-0.3218607,-1.983081e-08,0.8498447,-0.02474207,-0.6698989
4,1,0,0,0,0,0.7479827,0.3335088,3.498261e-08,0.1552378,0.02571763,0.2520173,-0.3335088,-3.498261e-08,-0.1552378,-0.02571763,-0.6501529
5,1,0,1,1,0,0.9376025,0.8034104,0.9953936,0.4666167,0.1131631,0.0623975,-0.8034104,0.004606351,0.5333833,-0.1131631,0.1341029
6,1,0,0,0,0,0.7479827,0.3335088,3.498261e-08,0.1552378,0.02571763,0.2520173,-0.3335088,-3.498261e-08,-0.1552378,-0.02571763,-0.6501529
